In [ ]:
# Uncomment the line below to install on Colab or similar
#! pip install git+https://github.com/monash-emu/renewal.git

In [ ]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, PANEL_SUBTITLES, plot_3d_spaghetti, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols
from emu_renewal.targets import StandardTarget

In [ ]:
# Specify fixed parameters and get calibration data
proc_update_freq = 14
init_time = 50
mys_data = pd.read_csv("https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv", index_col=0)["MYS"]
mys_data.index = pd.to_datetime(mys_data.index)
pop = 33e6
analysis_start = datetime(2021, 3, 1)
analysis_end = datetime(2021, 11, 1)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = mys_data.loc[analysis_start: analysis_end]
init_data = mys_data.loc[init_start: init_end]

In [ ]:
proc_fitter = CosineMultiCurve()
renew_model = RenewalModel(33e6, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), 50, init_data, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.Uniform(6.5, 10.5),
    "gen_sd": dist.Uniform(3.0, 4.6),
    "cdr": dist.Beta(4.0, 10.0),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.Uniform(8.0, 12.0),
    "report_sd": dist.Uniform(3.0, 6.0),
}

In [ ]:
calib_data = {
    "cases": StandardTarget(select_data, 0.1)
}
calib = StandardCalib(renew_model, priors, calib_data)

In [ ]:
#| output: false
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=calib.custom_init(radius=0.5))
mcmc = numpyro.infer.MCMC(kernel, num_chains=2, num_samples=1000, num_warmup=1000)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_dict(mcmc.get_samples(True))

In [ ]:
burn_in = 10
n_samples = 100
quantiles = [0.05, 0.5, 0.95]
idata_burnt = idata.sel(draw=slice(burn_in, None))
idata_sampled = az.extract(idata_burnt, num_samples=n_samples)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
spaghetti = get_spaghetti_from_params(calib, sample_params)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles)

In [ ]:
Markdown(renew_model.get_description())

In [ ]:
#| output: false
plot_spaghetti(spaghetti, select_data).update_layout(showlegend=False)

In [ ]:
patch_fig = plot_uncertainty_patches(quantiles_df, select_data, qual_colours.Plotly)
patch_fig.update_layout(showlegend=False)

In [ ]:
# Optional - requires kaleido
#| label: fig-calib
#| fig-cap: "Calibration to sample data from Malaysia"
# patch_fig.write_image("patch_fig.svg")

In [ ]:
# params_df.columns = ["name", "Lower limit", "Upper limit"]
# params_df.index = params_df["name"]
# params_df = params_df.drop(columns=["name"])
# params_df.index.name = None

In [ ]:
Markdown("### Calibration")

In [ ]:
calib.targets["cases"].dispersion_args

In [ ]:
Markdown(calib.get_description())

In [ ]:
# Markdown(params_df.to_markdown())

In [ ]:
# evidence_table = pd.DataFrame(index=params_df.index, columns=["Evidence"])
# evidence_table.loc[:, "Evidence"] = "To be populated [@cori2013]"
# Markdown(evidence_table.to_markdown())

In [ ]:
# plot_3d_spaghetti(spaghetti, ["susceptibles", "transmission potential"])

In [ ]:
summary = az.summary(idata)
summary = adjust_summary_cols(summary)
summary.index = summary.index.map(get_adjust_idata_index(renew_model))
Markdown(summary.to_markdown())

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);